# Lab 10: Word Embeddings
Thinking of using stuff from here
https://gist.github.com/mbednarski/da08eb297304f7a66a3840e857e060a0

## Janitorial Work

In [14]:
testCorpus = ["First of all, quit grinnin’ like an idiot. Indians ain’t supposed to smile like that. Get stoic.",
             "No. Like this. You gotta look mean, or people won’t respect you.",
              " people will run all over you if you don’t look mean.",
              "You gotta look like a warrior. You gotta look like you just came back from killing a buffalo.",
             "But our tribe never hunted buffalo. We were fishermen."
             "What? You wanna look like you just came back from catching a fish?",
             "This ain’t dances with salmon, you know. Thomas, you gotta look like a warrior."]

In [12]:
# Read in pubmed corpus into a text file
import glob
pubMedDataFolderPath = "data/ap_corpus/"
pubMedDataFiles = glob.glob(pubMedDataFolderPath + "*.txt")
pubMedCorpus = [""]*len(pubMedDataFiles)
for idx, pubMedDataPath in enumerate(pubMedDataFiles):
    with open(pubMedDataPath, "r") as pubMedFile:
        text = pubMedFile.read().strip()
        pubMedCorpus[idx] = text
print("{} pub med abstracts".format(len(pubMedCorpus)))

1767 pub med abstracts


In [13]:
# Read in the ap corpus
apTextFile = "data/ap.txt"
apCorpus = []
readText = False
with open(apTextFile) as apDataFile:
    for line in apDataFile:
        if readText:
            apCorpus.append(line.strip())
            readText = False
        if line == "<TEXT>\n":
            readText = True
print("{} ap articles".format(len(apCorpus)))

2246 ap articles


In [39]:
import string
def removePunctuation(myStr):
    excludedCharacters = string.punctuation + "’"
    newStr = "".join(char for char in s if char not in excludedCharacters)
    return(newStr)
def tokenize_corpus(corpus):
    tokens = [removePunctuation(x).split() for x in corpus]
    return tokens

apCorpusTokenized = tokenize_corpus(apCorpus)
pubMedCorpusTokenized = tokenize_corpus(pubMedCorpus)
testCorpusTokenized = tokenize_corpus(testCorpus)

In [41]:
def extractVocabMappers(tokenizedCorpus):
    vocabulary = []
    for sentence in tokenizedCorpus:
        for token in sentence:
            if token not in vocabulary:
                vocabulary.append(token)

    word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
    idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}
    return(word2idx, idx2word)

word2Idx_ap, idx2Word_ap = extractVocabMappers(apCorpus)
word2Idx_pubMed, idx2Word_pubMed = extractVocabMappers(pubMedCorpus)
word2Idx_test, idx2Word_test = extractVocabMappers(testCorpus)

In [38]:
import string
s = "First of all, quit grinnin’ like an idiot. Indians ain’t supposed to smile like that. Get stoic."
excludedCharacters = string.punctuation + "’"
s = "".join(char for char in s if char not in excludedCharacters)
s

'First of all quit grinnin like an idiot Indians aint supposed to smile like that Get stoic'

In [36]:
str.maketrans()

'First of all quit grinnin like an idiot Indians aint supposed to smile like that Get stoic'

In [16]:
testCorpusTokenized

[['First',
  'of',
  'all,',
  'quit',
  'grinnin’',
  'like',
  'an',
  'idiot.',
  'Indians',
  'ain’t',
  'supposed',
  'to',
  'smile',
  'like',
  'that.',
  'Get',
  'stoic.'],
 ['No.',
  'Like',
  'this.',
  'You',
  'gotta',
  'look',
  'mean,',
  'or',
  'people',
  'won’t',
  'respect',
  'you.'],
 ['people',
  'will',
  'run',
  'all',
  'over',
  'you',
  'if',
  'you',
  'don’t',
  'look',
  'mean.'],
 ['You',
  'gotta',
  'look',
  'like',
  'a',
  'warrior.',
  'You',
  'gotta',
  'look',
  'like',
  'you',
  'just',
  'came',
  'back',
  'from',
  'killing',
  'a',
  'buffalo.'],
 ['But',
  'our',
  'tribe',
  'never',
  'hunted',
  'buffalo.',
  'We',
  'were',
  'fishermen.What?',
  'You',
  'wanna',
  'look',
  'like',
  'you',
  'just',
  'came',
  'back',
  'from',
  'catching',
  'a',
  'fish?'],
 ['This',
  'ain’t',
  'dances',
  'with',
  'salmon,',
  'you',
  'know.',
  'Thomas,',
  'you',
  'gotta',
  'look',
  'like',
  'a',
  'warrior.']]

## Word2Vec Implementation

In [ ]:
import numpy as np
import torch

## Exploring Word Embeddings

## How Domains Affect Word Embeddings